In [1]:
import argparse
from dataclasses import dataclass
from functools import partial

import numpy as np
from tqdm import tqdm

import jax
from jax.experimental.maps import xmap
import jax.numpy as jnp

import flax
from flax import linen as nn

from tokenizers import Tokenizer

In [2]:
@jax.custom_vjp
def f_psum(x):
    return x

def f_psum_fwd(x):
    return f_psum(x), None

def f_psum_bwd(_, g):
    return jax.lax.psum(g, "shard"),

f_psum.defvjp(f_psum_fwd, f_psum_bwd)

In [3]:
@jax.custom_vjp
def g_psum(x):
    return jax.lax.psum(x, "shard")

def g_psum_fwd(x):
    return g_psum(x), None

def g_psum_bwd(_, g):
    return g,

g_psum.defvjp(g_psum_fwd, g_psum_bwd)

In [9]:
# class EmbeddingShard(hk.Module):
#     def __init__(self, config, name=None):
#         super().__init__(name=name)
#         in_dim = config["n_vocab"]
#         out_dim = config["d_model"]
#         shards = config["cores_per_replica"]

#         assert in_dim % shards == 0

#         self.in_dim = in_dim
#         self.out_dim = out_dim
#         self.in_dim_per_shard = in_dim // shards

#         # embed_init = hk.initializers.TruncatedNormal(stddev=0.02)
#         # self.positional_embeddings = hk.get_parameter('pos_embs', [seq_length, self.out_dim_per_shard], init=embed_init)
#         self.proj = hk.Linear(self.out_dim, w_init=hk.initializers.TruncatedNormal(stddev=1 / np.sqrt(in_dim)))

#     def __call__(self, x, dtype=jnp.bfloat16):
#         shard_start_index = jax.lax.axis_index('shard') * self.in_dim_per_shard

#         input_onehot = jax.nn.one_hot(x - shard_start_index, self.in_dim_per_shard)
#         proj_out = self.proj(input_onehot)

#         # all_pos_embed = jax.lax.all_gather(self.positional_embeddings, 'shard')
#         # all_pos_embed = hk.Flatten()(jnp.transpose(all_pos_embed, (1, 0, 2)))

#         return g_psum(proj_out)  # + all_pos_embed

class EmbeddingMP(nn.Module):
    config: dataclass

    def setup(self):
        self.embedding = self.param('embedding', nn.initializers.variance_scaling(1.0, 'fan_in', 'normal', out_axis=0), (self.config.vocab_size // self.config.mp, self.config.d_model), jnp.float32)

    def __call__(self, x):
        start_idx = jax.lax.axis_index('shard') * self.config.vocab_size // self.config.mp

        embedding = self.embedding[x - start_idx]

        return g_psum(embedding)

    # TODO: fuse output with crossentropy for efficiency
    def attend(self, x):
        start_idx = jax.lax.axis_index('shard') * self.config.vocab_size // self.config.mp

        proj = jnp.dot(x - start_idx, self.embedding.T)

        proj = jax.lax.all_gather(proj, "shard").transpose(1, 2, 0, 3)
        proj = proj.reshape(proj.shape[0], proj.shape[1], -1)
        print(proj.shape)

        return proj

In [10]:
def shard(xs):
    return jax.tree_map(
        lambda x: x.reshape((jax.device_count(), -1) + x.shape[1:]), xs)


class MultiHeadSelfAttention(nn.Module):
    config: dataclass

    @nn.compact
    def __call__(self, x, mask):
        batch, seq_len, d_model = x.shape

        d_model_mp = d_model // config.mp
        d_k = d_model_mp // config.n_heads
        
        x = f_psum(x)

        q = nn.Dense(d_model_mp)(x)
        k = nn.Dense(d_model_mp)(x)
        v = nn.Dense(d_model_mp)(x)

        # batch x n_heads x seq_len x d_k
        q = q.reshape((-1, seq_len, config.n_heads, d_k)).transpose((0, 2, 1, 3))
        k = k.reshape((-1, seq_len, config.n_heads, d_k)).transpose((0, 2, 1, 3))
        v = v.reshape((-1, seq_len, config.n_heads, d_k)).transpose((0, 2, 1, 3))

        a = jnp.matmul(q, k.transpose((0, 1, 3, 2))) / jnp.sqrt(d_k)

        mask = jnp.where(mask, 0, -jnp.inf)
        a += mask

        a = nn.softmax(a, axis=-1)
        a = jnp.matmul(a, v)

        # batch x seq_len x n_heads x d_k
        # batch x seq_len x d_model_mp
        a = a.transpose((0, 2, 1, 3)).reshape(-1, seq_len, d_model_mp)

        o = nn.Dense(d_model)(a)
        o = g_psum(o)

        return o



class MLP(nn.Module):
    config: dataclass

    @nn.compact
    def __call__(self, x):
        x = f_psum(x)

        x = nn.Dense((self.config.d_model // self.config.mp) * 4)(x)
        x = nn.gelu(x)
        x = nn.Dense(self.config.d_model)(x)

        x = g_psum(x)

        return x


class Block(nn.Module):
    config: dataclass

    @nn.compact
    def __call__(self, x, mask):

        x = x + MultiHeadSelfAttention(self.config)(nn.LayerNorm()(x), mask)
        x = x + MLP(self.config)(nn.LayerNorm()(x))

        return x


class GPT2(nn.Module):
    config: dataclass

    @nn.compact
    def __call__(self, x):
        seq_len = x.shape[-1]

        position_ids = jnp.arange(start=0, stop=seq_len, step=1)
        mask = jnp.triu(
            jnp.ones((1, seq_len, seq_len)), k=1) == 0

        # content_embedding = nn.Embed(self.config.vocab_size, self.config.d_model)
        content_embedding = EmbeddingMP(self.config)
        x = content_embedding(x) + nn.Embed(self.config.max_seq_len, self.config.d_model)(position_ids)

        for _ in range(self.config.n_layers):
            x = Block(config)(x, mask)

        x = nn.LayerNorm()(x)
        x = content_embedding.attend(x)

        return x

In [11]:
@dataclass
class Config:
    fast: bool = False

    mp: int = 1
    dp: int = 1

    batch_size: int = 256
    epochs: int = 30
    sample_len: int = 16

    max_seq_len: int = 128
    n_layers: int = 2
    vocab_size: int = 32768
    d_model: int = 768
    n_heads: int = 8
config = Config

In [12]:
tokenizer = Tokenizer.from_file('./tokenizer.json')
with open('./wikitext-2-raw/wiki.train.raw', 'r') as f:
    text = f.read()
tokenized = tokenizer.encode(text)
batches = jnp.array(tokenized.ids)

In [13]:
@partial(xmap, in_axes=(["shard", ...], ["batch", ...]), out_axes=["shard", ...])
def initialize(rng, init_batch):
    variables = GPT2(Config).init({'params': rng}, init_batch)
    optimizer = flax.optim.Adam(learning_rate=1e-4, beta1=0.5, beta2=0.9).create(variables)

    return optimizer

def loss_fn(variables, batch):
    x = batch[:, :-1]
    y = batch[:, 1:]
    y = jax.nn.one_hot(y, config.vocab_size)

    y_hat = GPT2(config).apply(variables, x)

    loss = jnp.sum(y * jax.nn.log_softmax(y_hat, axis=-1), axis=-1)
    return -jnp.mean(loss)

@partial(xmap, in_axes=(["shard", ...], ["batch", ...]), out_axes=(["shard", ...], ["batch", ...]))
def train_step(optimizer, batch):
    loss, grad = jax.value_and_grad(loss_fn)(optimizer.target, batch)

    loss = jax.lax.pmean(loss, axis_name=['shard'])
    grad = jax.lax.pmean(grad, axis_name=['batch'])

    optimizer = optimizer.apply_gradient(grad)

    return optimizer, loss

@partial(xmap, in_axes=(["shard", ...], ["batch", ...]), out_axes=(["batch", ...]))
def eval_step(optimizer, batch):
    loss = loss_fn(optimizer.target, batch)
    loss = jax.lax.pmean(loss, axis_name='shard')

    return loss

In [14]:
rng = jax.random.PRNGKey(42)

rngs = jax.random.split(rng, num=config.mp)
init_batches = jnp.zeros((config.dp, 1, 128), dtype=int)

optimizer = initialize(rngs, init_batches)

(1, 128, 32766)


AssertionError: batcher called with wrong axis name

In [ ]:
x = init_batches

In [ ]:
optimizer, loss = train_step(optimizer, x)

In [ ]:
loss

In [32]:
loss = eval_step(optimizer, x)

In [33]:
loss

DeviceArray([1.674041, 1.674041], dtype=float32)